<a href="https://colab.research.google.com/github/CG0718/Documentation-Generator-agent/blob/main/Documentation_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U google-generativeai requests

In [3]:
import os
import subprocess
import shutil
import tempfile
import requests
import json
import google.generativeai as genai

# --- 1. CONFIGURATION ---
SCALEDOWN_API_KEY = "SCALEDOWN_API_KEY"
GEMINI_API_KEY = "GEMINI_API_KEY"
REPO_URL = "REPO_URL"

# Setup Gemini
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel('gemini-2.5-flash') # Using Flash for speed/efficiency

def document_repository(repo_url):
    # 1. Clone the Repo
    with tempfile.TemporaryDirectory() as temp_dir:
        print(f"📥 Cloning {repo_url}...")
        try:
            subprocess.check_call(['git', 'clone', repo_url, temp_dir], stderr=subprocess.STDOUT)
        except:
            return "Error: Could not clone repo. Is the URL correct?"

        # 2. Collect Python and Markdown files
        full_codebase = ""
        for root, _, files in os.walk(temp_dir):
            for file in files:
                if (file.endswith(".py") or file.endswith(".java")) and ".git" not in root:
                    with open(os.path.join(root, file), 'r', encoding='utf-8', errors='ignore') as f:
                        full_codebase += f"\n\n# --- FILE: {file} ---\n" + f.read()

    if not full_codebase:
        return "Error: No code files found to document."

    # 3. Compress via ScaleDown
    print(f"⚡ Compressing codebase ({len(full_codebase)} characters)...")
    sd_url = "https://api.scaledown.xyz/compress/raw/"
    sd_payload = {
        "context": full_codebase,
        "prompt": "System Role: You are an Advanced Architectural Reasoning Agent. Your goal is to deconstruct a codebase to get all documentation relevant code and logic",
        "scaledown": {"rate": "auto"}
    }
    sd_headers = {'x-api-key': SCALEDOWN_API_KEY, 'Content-Type': 'application/json'}

    try:
        sd_res = requests.post(sd_url, headers=sd_headers, data=json.dumps(sd_payload), timeout=60)
        skeleton = sd_res.json().get("compressed_prompt", "Could not compress.")
    except Exception as e:
        return f"ScaleDown Error: {e}"

    # 4. Generate Docs via Gemini
    print("🤖 Gemini is analyzing the compressed code...")

    system_instruction = (
        "You are a Senior Staff Technical Writer specializing in Developer Experience (DX). Your task is to transform the provided code skeleton into a comprehensive, production-grade README.md."
    )

    # Combine instruction and data for Gemini
    prompt = f"""{system_instruction}### Instructions & Deliverables:
1. **Executive Summary:** Define the core value proposition and "The Why" behind this project.
2. **System Architecture & Domain Mapping:** Create a clear table mapping files to their respective domains (e.g., Persistence/DB, API/Backend, UI/Frontend, Middleware).
3. **Data Flow Analysis:** Describe the lifecycle of a request, from the entry point to storage and back to the user.
4. **Logic Deep-Dive:** Explain the primary business logic flow. Identify the "Entry Point" file and the "Controller" logic.
5. **Technical Stack:** List the technologies used based on the imports and file extensions found in the skeleton.

### Formatting Requirements:
- Use clean Markdown with a clear heading hierarchy (H1, H2, H3).
- Use Mermaid.js syntax to create a flowchart of the data flow.
- Ensure all code snippets in the README are syntactically highlighted.
- Tone: Professional, concise, and technical.

### Output:
Return ONLY the Markdown content for the README.md.
\n\nCODE SKELETON:\n{skeleton}"""

    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"Gemini Error: {e}"

# --- EXECUTION ---
if __name__ == "__main__":
    if "YOUR_" in GEMINI_API_KEY:
        print("❌ Please add your API keys first!")
    else:
        documentation = document_repository(REPO_URL)
        print("\n" + "="*50 + "\n")
        print(documentation)

        with open("README.md", "w") as f:
            f.write(documentation)
        print("\n✅ Documentation saved to 'RAG_RealEstate_Gemini_Docs.md'")

📥 Cloning https://github.com/Utkarsh-Sharma-107/DocumentationGenerator.git...


Error: No code files found to document.

✅ Documentation saved to 'RAG_RealEstate_Gemini_Docs.md'
